In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
from tkinter import ttk
from PIL import Image, ImageTk




In [2]:
# Function to load the trained model
def load_model():
    try:
        model = tf.keras.models.load_model('pneumonia_model.h5')
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

# Load the model
model = load_model()

In [3]:
def expert_system(prediction, age, symptoms):
   
    # Define key symptoms for pneumonia diagnosis
    key_symptoms = ["fever", "cough", "difficulty_breathing", "chest_pain"]
    
    # Count how many key symptoms the patient has
    symptom_count = sum(symptoms.get(symptom, False) for symptom in key_symptoms)
    
    # Apply rules based on prediction, age, and symptoms
    if prediction == "pneumonia":
        if age < 5:
            # High-risk group: children under 5
            if symptom_count >= 2:
                return "Refined Diagnosis: Pneumonia (Confirmed - High Risk Group)"
            else:
                return "Refined Diagnosis: Possible False Positive, Monitor Symptoms"
        elif age > 60:
            # High-risk group: elderly over 60
            if symptom_count >= 2:
                return "Refined Diagnosis: Pneumonia (Confirmed - Elderly)"
            else:
                return "Refined Diagnosis: Possible False Positive, Monitor Symptoms"
        else:
            # General population
            if symptom_count >= 3:
                return "Refined Diagnosis: Pneumonia (Confirmed)"
            else:
                return "Refined Diagnosis: Possible False Positive, Consider Further Testing"
    else:
        if symptom_count == 0:
            return "Refined Diagnosis: Normal (No Symptoms Present)"
        elif symptom_count >= 2:
            return "Refined Diagnosis: Possible False Negative, Further Testing Recommended"
        else:
            return "Refined Diagnosis: Normal, But Monitor Symptoms"


In [5]:

# Image Preprocessing
img_height = 150
img_width = 150

# Function to load the image and make prediction
def predict_pneumonia(image_path):
    try:
        # Open the image
        img = Image.open(image_path)

        # Convert grayscale to RGB if the image has only one channel
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # Resize image to match input size
        img = img.resize((img_height, img_width))

        # Convert image to numpy array and normalize pixel values
        img = np.array(img) / 255.0

        # Check if the image has 3 channels (RGB)
        if img.shape[-1] != 3:
            raise ValueError("Input image must have 3 channels (RGB)")

        # Add batch dimension for the model input
        img = np.expand_dims(img, axis=0)


        prediction = model.predict(img)

        # If prediction > 0.5, it's pneumonia
        if prediction > 0.5:
            return "pneumonia"
        else:
            return "normal"

    except Exception as e:
        messagebox.showerror("Error", f"Error processing the image: {str(e)}")
        return None


# Function to open file dialog and predict, then assess severity
def upload_and_predict():
    try:
        # Open file dialog for user to select an image
        file_path = filedialog.askopenfilename(
            title="Select a Chest X-ray Image",
            filetypes=[("Image files", "*.jpg;*.jpeg;*.png")]
        )

        if file_path:  # If user selected a file
            # Ensure the path is in a valid format for all OS
            file_path = file_path.replace("\\", "/")

            # Predict using the model
            result = predict_pneumonia(file_path)

            if result == "pneumonia":
                # If pneumonia is detected, ask for symptoms and age
                ask_for_details(result)
            else:
                messagebox.showinfo("Prediction Result", f"The result is: Normal\nNo pneumonia detected.")
        else:
            messagebox.showwarning("No file", "No image selected!")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

# Function to ask for patient age and symptoms
def ask_for_details(model_result):
    
    details_window = tk.Toplevel(window)
    details_window.title("Enter Patient Details")
    details_window.geometry("500x500")
    details_window.configure(bg="#f7f9fc")

    # Variables to store input
    age_var = tk.IntVar()
    fever_var = tk.BooleanVar()
    cough_var = tk.BooleanVar()
    difficulty_breathing_var = tk.BooleanVar()
    chest_pain_var = tk.BooleanVar()

    # Title Label
    tk.Label(
        details_window,
        text="Please Provide the Patient's Age & Check Symptoms if any",
        font=("Segoe UI", 16, "bold"),
        bg="#f7f9fc",
        fg="#2a6f97"
    ).pack(pady=20)

    # Age Input
    tk.Label(
        details_window,
        text="Age:",
        font=("Segoe UI", 14),
        bg="#f7f9fc"
    ).pack(pady=5)
    tk.Entry(details_window, textvariable=age_var, font=("Segoe UI", 14)).pack(pady=5)

    # Symptoms Checkboxes
    tk.Label(
        details_window,
        text="Symptoms:",
        font=("Segoe UI", 14),
        bg="#f7f9fc"
    ).pack(pady=10)

    symptoms_frame = tk.Frame(details_window, bg="#f7f9fc")
    symptoms_frame.pack()

    tk.Checkbutton(
        symptoms_frame, text="Fever", variable=fever_var, font=("Segoe UI", 12), bg="#f7f9fc"
    ).grid(row=0, column=0, sticky="w", padx=10)
    tk.Checkbutton(
        symptoms_frame, text="Cough", variable=cough_var, font=("Segoe UI", 12), bg="#f7f9fc"
    ).grid(row=1, column=0, sticky="w", padx=10)
    tk.Checkbutton(
        symptoms_frame, text="Difficulty Breathing", variable=difficulty_breathing_var, font=("Segoe UI", 12), bg="#f7f9fc"
    ).grid(row=2, column=0, sticky="w", padx=10)
    tk.Checkbutton(
        symptoms_frame, text="Chest Pain", variable=chest_pain_var, font=("Segoe UI", 12), bg="#f7f9fc"
    ).grid(row=3, column=0, sticky="w", padx=10)

    # Function to submit and provide a refined diagnosis with severity
    def submit_details():
        age = age_var.get()
        symptoms = {
            "fever": fever_var.get(),
            "cough": cough_var.get(),
            "difficulty_breathing": difficulty_breathing_var.get(),
            "chest_pain": chest_pain_var.get()
        }

        # Get the refined diagnosis (includes severity in the updated logic)
        refined_diagnosis = expert_system(model_result, age, symptoms)

        # Display the refined diagnosis with severity
        messagebox.showinfo("Diagnosis Result", refined_diagnosis)

        # Close the details window after displaying results
        details_window.destroy()


    # Submit Button
    tk.Button(
        details_window,
        text="Submit",
        command=submit_details,
        font=("Segoe UI", 14, "bold"),
        bg="#2a6f97",
        fg="#ffffff"
    ).pack(pady=20)

# Create the Tkinter window
window = tk.Tk()
window.title("CAD System for Pneumonia")
window.geometry("600x500")
window.configure(bg="#eaf4fc")

# Header Section
header_frame = tk.Frame(window, bg="#eaf4fc", pady=10)
header_frame.pack(fill="x")
header_label = tk.Label(
    header_frame,
    text="Pneumonia Detection Using Chest X-rays",
    font=("Segoe UI", 20, "bold"),
    bg="#eaf4fc",
    fg="#2a6f97"
)
header_label.pack()

# Instructions Section
instructions_frame = tk.Frame(window, bg="#ffffff", pady=20, padx=20, relief="groove", bd=2)
instructions_frame.pack(pady=20, padx=20, fill="both", expand=True)
instructions_label = tk.Label(
    instructions_frame,
    text="Upload a chest X-ray image to detect signs of pneumonia and get a refined diagnosis.",
    font=("Segoe UI", 14),
    bg="#ffffff",
    fg="#4a4a4a",
    wraplength=500,
    justify="center"
)
instructions_label.pack(pady=10)

# Upload Button
upload_button = tk.Button(
    instructions_frame,
    text="Upload Image",
    font=("Segoe UI", 16, "bold"),
    bg="#2a6f97",
    fg="#ffffff",
    command=upload_and_predict
)
upload_button.pack(pady=20)

# Footer Section
footer_frame = tk.Frame(window, bg="#eaf4fc", pady=5)
footer_frame.pack(fill="x", side="bottom")
footer_label = tk.Label(
    footer_frame,
    text="© 2024 Medical Expert | Sama El-Shorbagy",
    font=("Segoe UI", 10),
    bg="#eaf4fc",
    fg="#6c757d"
)
footer_label.pack()

# Run the Application
window.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 689ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
